In [212]:
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import csv
import operator
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder,LabelEncoder
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer

import math
import re

In [213]:
data = pd.read_csv('/Users/Sissi/Desktop/OA/amazon_co-ecommerce_sample.csv')
data.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [214]:
data = data.drop(['uniq_id', 'product_name','amazon_category_and_sub_category','customers_who_bought_this_item_also_bought','description','product_information','product_description','items_customers_buy_after_viewing_this_item','customer_questions_and_answers','sellers'], axis=1)

In [258]:
data.shape

(9979, 14)

In [215]:
data = data.dropna(subset=['customer_reviews'])

In [216]:
data.fillna(value = 'NAN')

,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,customer_reviews
0,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Worth Buying For The Pictures Alone (As Ever) ...
1,FunkyBuys,£16.99,NAN,2,1,4.5 out of 5 stars,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...
2,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,**Highly Recommended!** // 5.0 // 26 May 2015 ...
3,Hornby,£39.99,NAN,1,2,5.0 out of 5 stars,I love it // 5.0 // 22 July 2013 // By\n \n...
4,Hornby,£32.19,NAN,3,2,4.7 out of 5 stars,Birthday present // 5.0 // 14 April 2014 // By...
...,...,...,...,...,...,...,...
9995,Mattel,£22.95,5 new,3,3,5.0 out of 5 stars,Realistic // 5.0 // 31 Mar. 2014 // By\n \n...
9996,Star Wars,£39.99,NAN,1,3,4.0 out of 5 stars,... what I see my grandson us going to have fu...
9997,Olde Scotland Yard Ltd.,£43.99,3 new,1,3,5.0 out of 5 stars,Five Stars // 5.0 // 18 Dec. 2015 // By\n \...
9998,DC Comics,£49.81,3 new,1,3,5.0 out of 5 stars,The best sculpt in a while // 5.0 // 13 May 20...


In [217]:
objList = data.select_dtypes(include = "object").columns

In [218]:
col_vals = LabelEncoder()
for col in objList:
    if col == 'manufacturer':
        data['manufacturer_name'] = col_vals.fit_transform(data[col].astype(str))

In [219]:
for index, row in data.iterrows():
    score = row['average_review_rating']
    if score == score:
        value = re.findall("\d+\.?\d*", score)[0]
        tmp = float(value)
        frac = tmp - math.floor(tmp);
        if (frac < 0.5):
            data.loc[data.index == index, 'rate_num'] = math.floor(tmp)
        else:  
            data.loc[data.index == index, 'rate_num'] = math.ceil(tmp)

In [220]:
for index, row in data.iterrows():
    price = row['price']
    if price == price:
        value = re.findall("\d+\.?\d*", price)[0]
        data.loc[data.index == index, 'price_num'] = float(value)
        data['price_num'].astype(float)

In [221]:
for index, row in data.iterrows():
    stockVal = row['number_available_in_stock']
    if stockVal == stockVal:
        value = re.findall("\d+\.?\d*", stockVal)[0]
        data.loc[data.index == index, 'stock_num'] = float(value)
        data['stock_num'].astype(float)

In [222]:
for index, row in data.iterrows():
    question = row['number_of_answered_questions']
    if question == question:
        data.loc[data.index == index, 'question_ans_num'] = float(question)
        data['question_ans_num'].astype(float)

In [223]:
for index, row in data.iterrows():
    review = row['number_of_reviews']
    if review == review:
        value = review.replace(',','')
        #print(value)
        data.loc[data.index == index, 'review_num'] = float(value)
        data['review_num'].astype(float)

In [224]:
for index, row in data.iterrows():
    review = row['customer_reviews']
    list = review.split("//")
    comments = ''
    for idx, val in enumerate(list):
        if (idx % 4 == 0):   
            comments = comments + val
    data.loc[data.index == index, 'customer_comments'] = comments

In [106]:
data['customer_comments']

0       Worth Buying For The Pictures Alone (As Ever) ...
1       Four Stars  Very happy with the communication ...
2       **Highly Recommended!**  Simple & GREAT FUN fo...
3       I love it  I love it. Perfect with the earlier...
4       Birthday present  Bought this for my Grandson'...
                              ...                        
9995    Realistic  Very true to the Batman classic ser...
9996    ... what I see my grandson us going to have fu...
9997           Five Stars  arrived in excellent condition
9998    The best sculpt in a while  Okay, Hal Jordan f...
9999    Gold leader  1st class kit top marks to bandai...
Name: customer_comments, Length: 9979, dtype: object

In [225]:
data.head()

,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,customer_reviews,manufacturer_name,rate_num,price_num,stock_num,question_ans_num,review_num,customer_comments
0,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Worth Buying For The Pictures Alone (As Ever) ...,1026,5.0,3.42,5.0,1.0,15.0,Worth Buying For The Pictures Alone (As Ever) ...
1,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,798,5.0,16.99,NaN,1.0,2.0,Four Stars Very happy with the communication ...
2,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,**Highly Recommended!** // 5.0 // 26 May 2015 ...,2548,4.0,9.99,2.0,2.0,17.0,**Highly Recommended!** Simple & GREAT FUN fo...
3,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,I love it // 5.0 // 22 July 2013 // By\n \n...,1026,5.0,39.99,NaN,2.0,1.0,I love it I love it. Perfect with the earlier...
4,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Birthday present // 5.0 // 14 April 2014 // By...,1026,5.0,32.19,NaN,2.0,3.0,Birthday present Bought this for my Grandson'...


In [226]:
data_dict = {'manufacturer_name': data['manufacturer_name'],
             'price_num': data['price_num'],
            'stock_num': data['stock_num'],
            'review_num': data['review_num'],
            'answer_num': data['question_ans_num'],
            'customer_comments': data['customer_comments'],
            'rate_num': data['rate_num']}
useful_info_df = pd.DataFrame(data_dict)

In [227]:
useful_info_df['price_num'] = pd.to_numeric(useful_info_df['price_num'])
useful_info_df['stock_num'] = pd.to_numeric(useful_info_df['stock_num'])
useful_info_df['review_num'] = pd.to_numeric(useful_info_df['review_num'])
useful_info_df['answer_num'] = pd.to_numeric(useful_info_df['answer_num'])
useful_info_df['rate_num'] = pd.to_numeric(useful_info_df['rate_num'])

In [228]:
print (useful_info_df.isnull().sum())

manufacturer_name       0
price_num            1431
stock_num            2494
review_num              0
answer_num            764
customer_comments       0
rate_num                0
dtype: int64


In [229]:
num_cols = [col for col in useful_info_df.columns if useful_info_df[col].dtypes !='O']
print(num_cols)

['manufacturer_name', 'price_num', 'stock_num', 'review_num', 'answer_num', 'rate_num']


In [230]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median', fill_value=None)
for header in num_cols:
    imputer.fit(useful_info_df[[header]])
    useful_info_df[header] = imputer.transform(useful_info_df[[header]]).ravel()

In [231]:
print (useful_info_df.isnull().sum())

manufacturer_name    0
price_num            0
stock_num            0
review_num           0
answer_num           0
customer_comments    0
rate_num             0
dtype: int64


In [232]:
print("Number of rows per star rating:")
print(useful_info_df['rate_num'].value_counts())

Number of rows per star rating:
5.0    7580
4.0    2393
3.0       5
2.0       1
Name: rate_num, dtype: int64


In [233]:
useful_info_df.shape

(9979, 7)

In [234]:
useful_info_df.head()

,manufacturer_name,price_num,stock_num,review_num,answer_num,customer_comments,rate_num
0,1026.0,3.42,5.0,15.0,1.0,Worth Buying For The Pictures Alone (As Ever) ...,5.0
1,798.0,16.99,5.0,2.0,1.0,Four Stars Very happy with the communication ...,5.0
2,2548.0,9.99,2.0,17.0,2.0,**Highly Recommended!** Simple & GREAT FUN fo...,4.0
3,1026.0,39.99,5.0,1.0,2.0,I love it I love it. Perfect with the earlier...,5.0
4,1026.0,32.19,5.0,3.0,2.0,Birthday present Bought this for my Grandson'...,5.0


In [235]:
useful_info_df['customer_comments']

0       Worth Buying For The Pictures Alone (As Ever) ...
1       Four Stars  Very happy with the communication ...
2       **Highly Recommended!**  Simple & GREAT FUN fo...
3       I love it  I love it. Perfect with the earlier...
4       Birthday present  Bought this for my Grandson'...
                              ...                        
9995    Realistic  Very true to the Batman classic ser...
9996    ... what I see my grandson us going to have fu...
9997           Five Stars  arrived in excellent condition
9998    The best sculpt in a while  Okay, Hal Jordan f...
9999    Gold leader  1st class kit top marks to bandai...
Name: customer_comments, Length: 9979, dtype: object

In [190]:
useful_info_df.shape

(9979, 8)

In [236]:
import pandas as pd
import numpy as np
import nltk
import re

In [237]:
# To lower case
useful_info_df['pre_process_comments'] = useful_info_df['customer_comments'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

In [238]:
from bs4 import BeautifulSoup
# Get rid of URLs HTMLs
useful_info_df['pre_process_comments'] =useful_info_df['pre_process_comments'].apply(lambda x: BeautifulSoup(x).get_text())
import re
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x: re.sub(r"//", "", x))

In [239]:
def contractions(s):
    s = re.sub(r"won’t", "will not",s)
    s = re.sub(r"would’t", "would not",s)
    s = re.sub(r"could’t", "could not",s)
    s = re.sub(r"\’d", " would",s)
    s = re.sub(r"can\’t", "can not",s)
    s = re.sub(r"n\’t", " not", s)
    s= re.sub(r"\’re", " are", s)
    s = re.sub(r"\’s", " is", s)
    s = re.sub(r"\’ll", " will", s)
    s = re.sub(r"\’t", " not", s)
    s = re.sub(r"\’ve", " have", s)
    s = re.sub(r"\’m", " am", s)
    return s
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x:contractions(x))

In [240]:
import nltk 
from nltk import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/Sissi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Sissi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Sissi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [241]:
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))

In [242]:
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x: re.sub(' +', ' ', x))

In [243]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

In [244]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
useful_info_df['pre_process_comments'] = useful_info_df['pre_process_comments'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [245]:
useful_info_df['pre_process_comments'][0]

'worth buying picture alone ever part magic growing boy buy given new hornby catalogue every year even included product previous year still got old one dating back somewhere day catalogue especially informative tell vintage rolling stock useful dedicating railway one particular era train company amazing detail fabulous photography amazing detail every credit photographer book worthy reference manual well sale brochure even passing interest hobby transported another time younger awe big train great purchase purchased behalf dad always asking look gauge engine online good buy look anytime would definitely buy next one arrived quickly perfect condition great catalogue everything really needed see offer hornby way train would minded included rrp well though collect glossy picture great nice still get collect glossy picture great nice still get catalog collect great catalogue great book extremely useful insight future christmas present useful useful info someonelike starting back hobby many

In [249]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = useful_info_df['pre_process_comments']
vectorizer = TfidfVectorizer(
                                #max_featrues = 5000,
                                min_df=0.002,  
                                ngram_range=(1,3)

)
vects=vectorizer.fit(corpus)
vects=vectorizer.transform(corpus)
print(vecs.shape)

(9979, 5161)


In [250]:
useful_info_df.shape

(9979, 8)

In [251]:
final_useful_info_df= pd.DataFrame(X.toarray(),columns=feature_names)
final_useful_info_df.head(100)


,aa,aaa,ability,able,able play,able use,absolute,absolutely,absolutely brilliant,absolutely delighted,...,youngster,youth,youtube,yr,yr old,yr old daughter,yr old grandson,yr old love,yr old son,yugioh
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.138744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
final_useful_info_df.shape

(9979, 5161)

In [276]:
useful_info_df.shape

(9979, 6)

In [277]:
#useful_info_df = useful_info_df.drop(columns=['customer_comments', 'pre_process_comments'])

In [286]:
final_useful_info_df.reset_index(drop=True, inplace=True)
useful_info_df.reset_index(drop=True, inplace=True)
result_df = pd.concat([final_useful_info_df,useful_info_df], axis=1)


In [287]:
result_df.head()

,aa,aaa,ability,able,able play,able use,absolute,absolutely,absolutely brilliant,absolutely delighted,...,yr old grandson,yr old love,yr old son,yugioh,manufacturer_name,price_num,stock_num,review_num,answer_num,rate_num
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1026.0,3.42,5.0,15.0,1.0,5.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,798.0,16.99,5.0,2.0,1.0,5.0
2,0.138744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2548.0,9.99,2.0,17.0,2.0,4.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1026.0,39.99,5.0,1.0,2.0,5.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1026.0,32.19,5.0,3.0,2.0,5.0


In [288]:
result_df.shape

(9979, 5167)

In [312]:
X = result_df.iloc[0:9979,0:5166]
X = np.asarray(X).astype('float32')
X

array([[ 0.       ,  0.       ,  0.       , ...,  5.       , 15.       ,
         1.       ],
       [ 0.       ,  0.       ,  0.       , ...,  5.       ,  2.       ,
         1.       ],
       [ 0.1387437,  0.       ,  0.       , ...,  2.       , 17.       ,
         2.       ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  3.       ,  1.       ,
         3.       ],
       [ 0.       ,  0.       ,  0.       , ...,  3.       ,  1.       ,
         3.       ],
       [ 0.       ,  0.       ,  0.       , ..., 31.       , 11.       ,
         3.       ]], dtype=float32)

In [331]:
y = result_df.iloc[0:9979,5166:5167]
y = LabelEncoder().fit_transform(y)
y = np.asarray(y).astype('int32').reshape((-1,1))
y

/Users/Sissi/opt/anaconda3/envs/pratt_ml/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[3],
       [3],
       [2],
       ...,
       [3],
       [3],
       [3]], dtype=int32)

In [332]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train,X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print("Train:",X_train.shape,y_train.shape,"Test: ",(X_test.shape,y_test.shape))

Train: (7983, 5166) (7983, 1) Test:  ((1996, 5166), (1996, 1))


In [339]:
X_train

array([[0., 0., 0., ..., 5., 1., 1.],
       [0., 0., 0., ..., 5., 1., 1.],
       [0., 0., 0., ..., 3., 1., 1.],
       ...,
       [0., 0., 0., ..., 5., 1., 1.],
       [0., 0., 0., ..., 4., 1., 1.],
       [0., 0., 0., ..., 5., 1., 1.]], dtype=float32)

In [340]:
y_train

array([[2],
       [2],
       [3],
       ...,
       [3],
       [2],
       [3]], dtype=int32)

In [352]:
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
# Define the keras model
model = Sequential()
model.add(Dense(8, input_dim=5166, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [355]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics=['accuracy'])

In [356]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 8)                 41336     
_________________________________________________________________
dropout_18 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_28 (Dense)             (None, 8)                 72        
_________________________________________________________________
dropout_19 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 45        
Total params: 41,453
Trainable params: 41,453
Non-trainable params: 0
_________________________________________________________________


In [359]:
model.fit(X_train,y_train,epochs=50,batch_size=128)

Epoch 1/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5844 - accuracy: 0.7604
Epoch 2/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5764 - accuracy: 0.7604
Epoch 3/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5684 - accuracy: 0.7604
Epoch 4/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5567 - accuracy: 0.7604
Epoch 5/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5523 - accuracy: 0.7602
Epoch 6/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5486 - accuracy: 0.7604
Epoch 7/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5603 - accuracy: 0.7604
Epoch 8/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5662 - accuracy: 0.7604
Epoch 9/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5508 - accuracy: 0.7604
Epoch 10/50
63/63 [==============================] - 0s 2ms/step - loss: 0.5366 - accuracy: 0.7604
Epoch 11/50
63/63 [

In [386]:
score_train = model.evaluate(X_train,y_train,batch_size=128)
print("%s: %.2f%%" % (model.metrics_names[1], score_train[1]*100))

63/63 [==============================] - 0s 2ms/step - loss: 0.3123 - accuracy: 0.7604
accuracy: 76.04%


In [387]:
score_test = model.evaluate(X_test,y_test,batch_size=128)
print("%s: %.2f%%" % (model.metrics_names[1], score_test[1]*100))

16/16 [==============================] - 0s 3ms/step - loss: 0.4656 - accuracy: 0.7565
accuracy: 75.65%
